In [1]:
import sys
from pathlib import Path
from datetime import datetime
import argparse
import json

import torch
from torchvision.utils import save_image
import torchvision

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle

from svox2 import *
from pyvox.models import Vox, Color
from pyvox.writer import VoxWriter

from importlib import reload as reload

reload(svox2)
from svox2 import *

#TODO> modify this:
sys.path.append("/workspace/svox2/opt")
from util.dataset import datasets
from util import config_util

# Our nice tools
sys.path.append("/workspace/aseeo-research")
import RLResearch.utils.depth_utils as du


In [3]:
%tb

data_dir = "/workspace/datasets/cactus"
exp_name = "std"
checkpoint_path = Path(data_dir)/"ckpt"/exp_name/"ckpt.npz"


# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

# Load arguments from json
json_config_path = Path(data_dir)/"ckpt"/exp_name/"args.json"
parser = argparse.ArgumentParser()
with open(str(json_config_path.resolve()), 'rt') as f:
        t_args = argparse.Namespace()
        t_args.__dict__.update(json.load(f))
        args = parser.parse_args(namespace=t_args)

# parser = argparse.ArgumentParser()
# args = parser.parse_args([])

dataset = datasets["nsvf"](
            data_dir,
            split="train",
            device=device,
            factor=1,
            n_images=None,
            **config_util.build_data_options(args))

grid = SparseGrid.load(str(checkpoint_path.resolve()))


SystemExit: 2

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"f3f65935-e7d7-4073-8651-763793a3c678" --shell=9002 --transport="tcp" --iopub=9004 --f=/tmp/tmp-180333PlzyQAbRzwr6.json


SystemExit: 2

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

## Single camera position
img_id = 5
c2w = dataset.c2w[img_id].to(device = device)
cam = svox2.Camera(c2w,
                    dataset.intrins.get('fx', img_id),
                    dataset.intrins.get('fy', img_id),
                    dataset.intrins.get('cx', img_id),
                    dataset.intrins.get('cy', img_id),
                    width=dataset.get_image_size(img_id)[1],
                    height=dataset.get_image_size(img_id)[0],
                    ndc_coeffs=dataset.ndc_coeffs)

print("Using thres", args.log_depth_map_use_thresh)
depth_img = grid.volume_render_depth_image(cam,
                                    args.log_depth_map_use_thresh if
                                    args.log_depth_map_use_thresh else None
                                )


## Export colored pointcloud to check in meshlab
